In [1]:
using MAT

fileIn = matopen("../data/IEEE9data.mat")
dset = read(fileIn)
close(fileIn)

# extract the data from the nested Dict structure
dataDict = dset["data"] # Access the outer dictionary
# Access the nested dictionary 
c = dataDict["c"]
pd = dataDict["p"]["d"]
pg_lb = dataDict["p"]["min"]
pg_ub = dataDict["p"]["max"]
qd = dataDict["q"]["d"]
qg_lb = dataDict["q"]["min"]
qg_ub = dataDict["q"]["max"]
G = dataDict["G"]
B = dataDict["B"]
v_lb = dataDict["v"]["min"]
v_ub = dataDict["v"]["max"]

9×1 Matrix{Float64}:
 1.1
 1.1
 1.1
 1.1
 1.1
 1.1
 1.1
 1.1
 1.1

In [21]:
using JuMP, MosekTools
using LinearAlgebra

N = 9 # IEEE-9 system

# Create a P_SDP with Mosek as the solver
P_SDP = Model(Mosek.Optimizer)
set_optimizer_attribute(P_SDP, "MSK_DPAR_INTPNT_TOL_REL_GAP", 1e-6)  # Adjust relative gap tolerance


# Define variables with correct indexing
@variable(P_SDP, pg_lb[n] <= pg[n=1:N] <= pg_ub[n])
@variable(P_SDP, qg_lb[n] <= qg[n=1:N] <= qg_ub[n])
@variable(P_SDP, E[1:N, 1:N], Symmetric)
@variable(P_SDP, F[1:N, 1:N], Symmetric)
@variable(P_SDP, H[1:N, 1:N])

# Assuming c, G, B, pd, and qd are defined somewhere in your code
# Define the objective function
@objective(P_SDP, Min, sum(c[n] * pg[n] for n in 1:N))

# Define the constraints
for n in 1:N
    @constraint(P_SDP, sum(G[n,m] * (E[n,m] + F[n,m]) for m in 1:N) - 
                        sum(B[n,m] * (H[n,m] - H[m,n]) for m in 1:N) == pg[n] - pd[n])
                        
    @constraint(P_SDP, -sum(B[n,m] * (E[n,m] + F[n,m]) for m in 1:N) - 
                        sum(G[n,m] * (H[n,m] - H[m,n]) for m in 1:N) == qg[n] - qd[n])

    @constraint(P_SDP, v_lb[n]^2 <= E[n,n] + F[n,n] <= v_ub[n]^2)
end

# Solution status: SLOW_PROGRESS
# the problem is badly scaled or otherwise ill-conditioned

# # Add the PSD constraint # 
# @constraint(P_SDP, [E H; H' F] >= 0, PSDCone())
@constraint(P_SDP, [E H; H' F] >= 1e-3 * Matrix{Float64}(I, 2N, 2N), PSDCone())


# println(P_SDP)
# Solve the P_SDP
optimize!(P_SDP)

# Check the status of the solution
status = termination_status(P_SDP)
println("Solution status: ", status)

# Get the optimal value of the objective function if the solution is optimal
if status == MOI.OPTIMAL
    optimal_value = objective_value(P_SDP)
    println("Optimal value: ", optimal_value)

    # Get the optimal values of the variables
    optimal_pg = value.(pg)
    println("Optimal pg: ", optimal_pg)
    optimal_qg = value.(qg)
    println("Optimal qg: ", optimal_qg)
    # Add similar lines for E, F, and H if needed
else
    println("The problem does not have an optimal solution.")
end




Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 198             
  Cones                  : 0               
  Scalar variables       : 189             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 21
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective s

(b) why $\min(P_{SDP}) \geq \min P_{SOCP}$?

Since SOCP is subset of SDP, SDP relax the problem further, which lead compromised solution. In the minimization problem, we get larger optimal value. 


In [5]:
using JuMP, MosekTools

N = 9 # IEEE-9 system

# Create a P_SDP with Mosek as the solver
P_SOCP = Model(Mosek.Optimizer)

# Define variables with correct indexing
@variable(P_SOCP, pg_lb[n] <= pg[n=1:N] <= pg_ub[n])
@variable(P_SOCP, qg_lb[n] <= qg[n=1:N] <= qg_ub[n])
@variable(P_SOCP, E[1:N, 1:N], Symmetric)
@variable(P_SOCP, F[1:N, 1:N], Symmetric)
@variable(P_SOCP, H[1:N, 1:N])

# Assuming c, G, B, pd, and qd are defined somewhere in your code
# Define the objective function
@objective(P_SOCP, Min, sum(c[n] * pg[n] for n in 1:N))

# Define the constraints
for n in 1:N
    @constraint(P_SOCP, sum(G[n,m] * (E[n,m] + F[n,m]) for m in 1:N) - 
                        sum(B[n,m] * (H[n,m] - H[m,n]) for m in 1:N) == pg[n] - pd[n])
                        
    @constraint(P_SOCP, -sum(B[n,m] * (E[n,m] + F[n,m]) for m in 1:N) - 
                        sum(G[n,m] * (H[n,m] - H[m,n]) for m in 1:N) == qg[n] - qd[n])

    @constraint(P_SOCP, v_lb[n]^2 <= E[n,n] + F[n,n] <= v_ub[n]^2)
end


X = @variable(P_SOCP, [1:2N, 1:2N])

# Constraint 1: Non-negative diagonal elements
for i in 1:2N
    @constraint(P_SOCP, X[i,i] >= 0)
end

# Constraints to define X using E, H, and F
for i in 1:N
    for j in 1:N
        @constraint(P_SOCP, X[i, j] == E[i, j]) # E block
        @constraint(P_SOCP, X[i, N+j] == H[i, j]) # H block
        @constraint(P_SOCP, X[N+i, j] == H[j, i]) # H^T block
        @constraint(P_SOCP, X[N+i, N+j] == F[i, j]) # F block
    end
end

# Diagonal constraints: X_ii >= 0
for i in 1:2N
    @constraint(P_SOCP, X[i, i] >= 0)
end

# Second-order cone constraints
for i in 1:2N
    for j in i+1:2N
        @constraint(P_SOCP, [X[i, i] + X[j, j], 2*X[i, j], X[i, i] - X[j, j]] in SecondOrderCone())
    end
end

println(P_SOCP)
# Solve the P_SDP
optimize!(P_SOCP)

# Check the status of the solution
status = termination_status(P_SOCP)
println("Solution status: ", status)

# Get the optimal value of the objective function if the solution is optimal
if status == MOI.OPTIMAL
    optimal_value = objective_value(P_SOCP)
    println("Optimal value: ", optimal_value)

    # Get the optimal values of the variables
    optimal_pg = value.(pg)
    println("Optimal pg: ", optimal_pg)
    optimal_qg = value.(qg)
    println("Optimal qg: ", optimal_qg)
    # Add similar lines for E, F, and H if needed
else
    println("The problem does not have an optimal solution.")
end


Min 500 pg[1] + 120 pg[2] + 100 pg[3]
Subject to
 -pg[1] + 17.36111111111111 H[4,1] - 17.36111111111111 H[1,4] = 0.0
 -qg[1] + 17.36111111111111 E[1,1] - 17.36111111111111 E[1,4] + 17.36111111111111 F[1,1] - 17.36111111111111 F[1,4] = 0.0
 -pg[2] + 16 H[8,2] - 16 H[2,8] = 0.0
 -qg[2] + 16 E[2,2] - 16 E[2,8] + 16 F[2,2] - 16 F[2,8] = 0.0
 -pg[3] + 17.064846416382252 H[6,3] - 17.064846416382252 H[3,6] = 0.0
 -qg[3] + 17.064846416382252 E[3,3] - 17.064846416382252 E[3,6] + 17.064846416382252 F[3,3] - 17.064846416382252 F[3,6] = 0.0
 -pg[4] + 3.3073789620253065 E[4,4] - 1.9421912487147268 E[4,5] - 1.3651877133105799 E[4,9] + 3.3073789620253065 F[4,4] - 1.9421912487147268 F[4,5] - 1.3651877133105799 F[4,9] - 17.36111111111111 H[4,1] + 17.36111111111111 H[1,4] + 10.510682051867931 H[5,4] + 11.60409556313993 H[9,4] - 10.510682051867931 H[4,5] - 11.60409556313993 H[4,9] = 0.0
 -qg[4] - 17.36111111111111 E[1,4] + 39.30888872611897 E[4,4] - 10.510682051867931 E[4,5] - 11.60409556313993 E[4,9] - 